<a href="https://colab.research.google.com/github/mattwilliams06/SimpsonsRule/blob/master/SimpsonsRuleDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simpson's Rule is a numerical integration technique that effectively reduces an integration to a linear equation. The general form of Simpson's Rule is:

$ S_n = \frac{\Delta x}{3}(f(x_1) + 4f(x_2) + 2f(x_3) + ... + 4f(x_{n-1}) + f(x_n)) $

The 4, 2, 4, 2... pattern for the interior points repeats. For Simpson's rule to work properly, the domain must be divided into an even number of intervals, that is $ (N-1) \% 2 = 0 $ where $\%$ is the modulo operator, and $N$ is the number of points in the domain. The pattern 1, 4, 2, ..., 2, 4, 1 comes from the Simpson's multipliers, which are 1, 4, 1. In the derivation of Simpson's Rule, as an example, the first three points, $f(x_1)$, $f(x_2)$, and $f(x_3)$ are multiplied by 1, 4, and 1 respectively. This is added to 1, 4, 1 multiplied by $f(x_3)$, $f(x_4)$, and $f(x_5)$. You can see that $f(x_3)$ is repeated in the sum, with multiplier one. This is true of each interior end point when all the points are grouped by 3's. Therefore, every interior point that has a multiplier of 1 appears twice. Each point with a multiplier of 4 appears once. This leads to the 1, 4, 2, 4... pattern.

This is important to point out for transforming Simpson's Rule into code. Creating the pattern from its components of 1, 4, 1 is easier than attempting to create the entire pattern based on the length of the domain.

We will begin by first importing the NumPy library, which will aid in the creation of arrays.

In [0]:
import numpy as np

As is customary with the Python development world, the NumPy library is imported under the alias "np" to make the code more readable.

At the end of this tutorial, we will assemble everything we've done into one function. But for now, let's create mini-functions to demonstrate the code and do what we want.

The first thing that we must do is ensure that the domain is divided into an even number of intervals. Let's first create an array by dividing the interval [0, 1] into 10.

In [0]:
h = 0.1
x = np.arange(0., 1.+h, h)

In [3]:
x

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

The ```arange()``` function takes the interval between the first two specified points and divides it into intervals of length h (here, 0.1). I added h to 1 at the end because ```arange()``` is exclusive at the right endpoint. Notice how x stops at 1 and not 1.1.

We want to check if the user's input array is contains an odd-number of points, or an even number of intervals. Let's convert x into points from a function (perhaps a set of offsets in Naval Architecture terms). Let's use $f(x) = y^3$ for illustration purposes. We will take advantage of Python's vectorized operations by just cubing all the points in x:

In [4]:
y = x**3
y

array([0.   , 0.001, 0.008, 0.027, 0.064, 0.125, 0.216, 0.343, 0.512,
       0.729, 1.   ])

Let's write a function to check if the input is divided into an even number of intervals:

In [0]:
def check_length(x):
  l = len(x) - 1
  if l % 2 == 0:
    print("Simpson's Rule may be used")
  else:
    print("Odd-number of intervals. Please input an array with an even number of intervals")

To check whether or not Simpson's Rule can be used with our array, let's call check_length on y:

In [6]:
check_length(y)

Simpson's Rule may be used


It passed the test (by design). In the function later, this test will be used to determine whether the program may continue, or break and return the error message to the user.

Of course, we could always ask the user to pass the interval length to us, or we can calculate it ourselves. Simpson's Rule can be applied to intervals that do not have a constant spacing, and I address this in a class method in my GitHub repo for Simpson's Rule, but it requires more input from the user, so for now we will assume a constant interval spacing.

Let's write a function to compute the interval spacing and return it by dividing the total interval by the number of intervals. Remember, there are 11 points, but 10 intervals. We do need the user to tell us how long the total interval is. Remember, we only have the function points in y. Alternatively, we could also have the array x, which is where each y point corresponds to.

Let's do the case where the user tells us the length:

In [0]:
def calc_interval(y, L):
  l = L / (len(x) - 1)
  return l

In [8]:
calc_interval(y, 1.)

0.1

We can see the function correctly computed the interval. 

We now need to determine how many "sets" of the Simpson's multipliers ```[1, 4, 1]``` we will need, but we want the program to be nimble enough to be able to correctly perform this computation for any input.

You can see that with an overlap of two values per each interior node, in our particular problem, there are 5 groups of points corresponding to 5 sets of Simpson's multipliers. Grouping the points, they are [$f(x_1)$, $f(x_2)$, $f(x_3)$], [$f(x_3)$, $f(x_4)$, $f(x_5)$], [$f(x_5)$, $f(x_6)$, $f(x_7)$], [$f(x_7)$, $f(x_8)$, $f(x_9)$], and [$f(x_9)$, $f(x_{10})$, $f(x_{11})$]. If we were to add another 2 points, we would need a 6th grouping, and so on. In general, the number of groupings is equal to the number of intervals divided by two. In out example, there are 10 intervals, so we need 5 groupings. 

To create a one-dimensional array of the appropriate Simpson's multipliers, we will begin with a two-dimensional array of zeros, one row per grouping and one column per data point. In our example, this will be a 5x11 array. We will then fill the array in the proper positions with the ```[1, 4, 1]``` multipliters in each row. Then, if we collapse the array across the rows, this will create the proper pattern.

We will first instantiate an array of zeros using the proper number of rows and columns. We'll write a function to do so:

In [0]:
def create_array(x):
  n_cols = len(x)               # Number of points
  n_rows = (n_cols - 1) // 2    # Number of intervals, integer division by 2
  mat = np.zeros((n_rows, n_cols))
  return mat

In [10]:
simps = create_array(x)
simps

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

This function works as expected. Now, let us fill the array with the multipliers using the proper overlaps. We can iterate down the rows in a ```for``` loop, and iterate down the columns in a nested ```for``` loop. In each row, the starting column is the row index * 2. For example, in the first row, the starting column is $0*2= 0$. Recall that Python indexing begins at zero. In the second row, the beginning column will be $2*1=2$.

Let's create a function to do this. We can call ```create_array()``` within this function to get the matrix of zeros first. Then, our function can figure out the number of rows and columns by calling the ```shape``` attribute of the returned array. For example, the shape of the array ```simps``` is:

In [11]:
simps.shape

(5, 11)

We can see that it is a 5x11 array. To get the number of rows specifically, we can call the first element of the shape:

In [12]:
simps.shape[0]

5

We will leverage this to get the number of rows and columns without "hard-coding". We will also set the values in the zeros matrix by using the index notation ```array[row, column]```.

In [0]:
def mult_array(x, func):
  mults = np.array([1., 4., 1.])
  zero_array = func(x)
  for i in range(zero_array.shape[0]):
    for j in range(mults.shape[0]):
      zero_array[i, 2*i+j] = mults[j]
  return zero_array

In [14]:
mult_array(x, create_array)

array([[1., 4., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 4., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 4., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 4., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 4., 1.]])

You can see I created an argument in ```mult_array()``` to pass in a function. The result is what we want: The ```[1, 4, 1]``` multipliers at each associated group of points, and zeroes everywhere else. In order to get the 1D array of multipliers in the pattern ```[1, 4, 2, 4, ... 4, 2, 1]```, we will collapse this array by summing along axis zero, which is down the columns. Let's modify ```mult_array()``` to do this.

In [0]:
def mult_array(x, func):
  mults = np.array([1., 4., 1.])
  zero_array = func(x)
  for i in range(zero_array.shape[0]):
    for j in range(mults.shape[0]):
      zero_array[i, 2*i+j] = mults[j]
  simps_mults = zero_array.sum(axis=0)
  return simps_mults

In [16]:
mult_array(x, create_array)

array([1., 4., 2., 4., 2., 4., 2., 4., 2., 4., 1.])

Now, to put it all together, we need to multiply each point by its appropriate Simpson's multiplier, sum them all together, and multiply by $\frac{\Delta x}{3}$ as shown in the equation for $S_n$. We can again take advantage of Python's vectorized operations and not worry about using a loop to do this.

In [0]:
def calc_integral(h, points, mults):
  return h/3 * np.sum(points*mults)

In [18]:
s_mults = mult_array(x, create_array)
calc_integral(0.1, y, s_mults)

0.25

Here, I stored the mutlipliers in a variable ```s_mults``` to pass to the function, and gave it the interval length of 0.1

The integral of $x^3$ from 0 to 1 is 0.25, so in this case, Simpson's Rule has returned the exact answer, in the precision of the returned value by the computer. What's stored in memory is likely not the exact answer, so just keep this in mind.

We can now combine these functions into a single function to intake the points (or offsets) and return the area. Keep in mind that in the case of offsets, the returned area will need to be multiplied by 2. You should specify what the function will return in the function's documentation.

In [0]:
def simpsons_first_rule(offsets, L):
  ''' Use Simpson's First Rule to integrate the supplied points (offset) over the domain L.
  Inputs
  ------
  Offsets: 1D array -- array of points comprising the shape to be integrated
  L: float -- length of the domain. 

  Output
  ------
  area: The result of the integral. This value must be multiplied by two for ship applications.
  '''
  import numpy as np

  y = np.asarray(offsets)     # Convert to array in case the user passes a list
  n_points = len(y)
  if (n_points - 1) % 2 != 0:
    print('Odd number of intervals. Please pass an array with an even number of intervals (odd number of points).')
    return
    # Return statement breaks out of the function if there are an odd number of intervals
  h = L/(n_points - 1)              # Length of each interval
  n_rows = (y.shape[0] -1) // 2     # Get the number of rows for the zeros matrix
  n_cols = n_points                 # Refactor

  zero_matrix = np.zeros((n_rows, n_cols))
  mults = np.array([1., 4., 1.])
  for i in range(n_rows):
    for j in range(mults.shape[0]):
      zero_matrix[i, 2*i + j] = mults[j]

  simps_mults = zero_matrix.sum(axis=0)

  area = h/3 * np.sum(y*simps_mults)
  
  return area



You can see that all the functionality of the previous individual functions are there. It's possible to write a function that calls other functions as you've seen, but you want to choose the coding style that makes the code the most readable. For a short function like this, it's best to keep it all under one roof. That way, the reader can trace the code from start to finish easily and understand what's going on under the hood. Writing documentation and comments is also important for code readibility. If a piece of code is confusing, it's best to add comments and add "stuff" to the code than to leave the comment out and confuse the reader. Passing a function to a function and calling from within the function is slick, and it can make the code more readable, but the reader has to keep track of which function has been passed in and when it is called.

In [22]:
simpsons_first_rule(y, 1.)

0.25

We can see that this gives the correct result.

Say that you wanted to compute the transverse moment of inertia at the waterplane for a ship. Let's use a barge so that we can verify the results. For a rectangle of breadth ```b``` and length ```h```, the moment of inertia, or second moment of area, about an axis bisecting the rectangle through the breadth is $I_{bb} = \frac{bh^3}{12}$. For an infinitessimal rectangle, the moment of inertia about an axis __through the bottom of the rectangle (y = 0)__ is 

$$dI = \frac{y^3}{3}dx$$

In Naval Architecture, _y_ is the offset, or half-breadth distance of the hull from centerline. The x-direction is the longitudinal axis, and so you must integrate along the length of the waterline.

to use Simpson's rule in order to integrate this, we must take our half-breadth offsets, cube them, divide by three, and then pass that array to the function above remembering to multiply by 2 either before or after the result comes back from the function. 

Let's say the barge is 10 meters long and 2.5 meters in breadth. The transverse moment of inertia is therefore:

$$I_T = \frac{10m * (2.5m)^3}{12} = 13.02m^4$$

Let's prepare an array to pass to ```simpsons_first_rule```. We want all the offsets to be 1.25 (the half-breadth). We can choose how many there are. Let's do one every meter, beginning at the forward perpendicular and ending at the aft perpendicular. We want 11 points total, so that there are 10 intervals. Conveniently, we can use Numpy's ```ones``` function, which returns an array of ones of a specified shape. We'll just tell it to give us 11 points. We will then mutliply the array by 1.25 to give us the offsets we want.

In [29]:
offsets = np.ones(11)*1.25
offsets

array([1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25])

Continuing with our data preprocessing, we need to cube the values, and then divide by three. Let's multiply by 2 before passing it in to the function, so we will actually multiply by two-thirds.

In [30]:
offsets = offsets**3
offsets *= 2/3
offsets

array([1.30208333, 1.30208333, 1.30208333, 1.30208333, 1.30208333,
       1.30208333, 1.30208333, 1.30208333, 1.30208333, 1.30208333,
       1.30208333])

In Python (and many other languages), ```offsets *= 2/3``` is shorthand for ```offsets = offsets * 2/3```. We are now ready to pass the information to ```simsons_first_rule```.

In [31]:
simpsons_first_rule(offsets, 10)

13.020833333333332

We can see that we obtained the expected result. We actually never checked that our function would break and print the error message if the input array had the wrong shape. Let's test that now. If the number of points is even, the function should print the message and return nothing, which simply exits out of the function and doesn't execute any more of the code.

In [36]:
test = np.array([1., 2., 3., 4.])
simpsons_first_rule(test, 1)

Odd number of intervals. Please pass an array with an even number of intervals (odd number of points).


This works as expected.